In [1]:
from selenium import webdriver
import time

driver = webdriver.Chrome('C:/Users/suruki/Desktop/suruki2021/chromedriver.exe')


In [2]:
url ='https://www.instagram.com'
driver.get(url)
time.sleep(2) #주석

In [3]:
# 내 인스타그램 로그인, 자동으로 이메일 주소 넣기 
email='hanpagui@naver.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)


In [11]:
# 검색어 입력 실행어 
word = '종로여행'
url = 'https://www.instagram.com/explore/tags/'+ word 
driver.get(url)

In [12]:
# 특정게시물 클릭하기 
first = driver.find_element_by_css_selector('div._9AhH0') #해당페이지 관리자 모드 들어가서 복사해오기,#element 주의 (not elements)
first.click()
time.sleep(1)

In [13]:
import re
from bs4 import BeautifulSoup
import unicodedata

In [14]:
# 현재 페이지 html 정보 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

# 본문 내용 가져오기
try:
    content = soup.select('div.C4VMK > span')[0].text
    content = unicodedata.normalize('NFC', content)
except:
    content = ''

# 본문 내용 해시태그 가져오기
tags = re.findall(r'#[^\s#,\\]+', content)
# findall : 정규식과 매치되는 모든 문자열을 리스트로 리턴
# r'패턴문자열' -> content에서 패턴문자열을 찾아 리스트로 리턴
# #으로 시작해서 
# ^ : 뒤에 문자들이 없어야 함 (whitespace, #, 콤마(,), 역슬래쉬(\)) + 글자들
# \s : whitespace / 


# 작성일자 정보
date = soup.select('time.FH9sR.Nzb55')[0]['datetime'][:10]

# 좋아요
try:
    like = soup.select('div.Nm9Fw > a > span')[0].text
except:
    like = 0

try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''

data = [content, date, like, place, tags]

data

['💚인왕산 성공적💚.최근 들어 제일 행복했던 순간.등산 후 마스크 살짝 벗고 숨쉴 때그렇게 상쾌하며 행복할 수가 없더라고요👍찐 행복한 웃음😊.정말 인왕산 찐 추천입니다!특히 오후 등산 추천👍😊👍.같이 가실 분 있나 혹쉬?.#서울등산#서울산#인왕산#인왕산정상#인왕산자락길#인왕산둘레길#인왕산숲길#인왕산등산#서울사진맛집#종로여행#국내여행#여행에미치다',
 '2020-04-26',
 '480',
 '인왕산',
 ['#서울등산',
  '#서울산',
  '#인왕산',
  '#인왕산정상',
  '#인왕산자락길',
  '#인왕산둘레길',
  '#인왕산숲길',
  '#인왕산등산',
  '#서울사진맛집',
  '#종로여행',
  '#국내여행',
  '#여행에미치다']]

In [8]:
#다음페이지 누르기 

right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
right.click()
time.sleep(1)


In [15]:
# 함수로 만들기
def get_content(driver):
    # 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ''
    
    # 본문 내용 해시태그 가져오기
    tags = re.findall(r'#[^\s#,\\]+', content)
    # findall : 정규식과 매치되는 모든 문자열을 리스트로 리턴
    # r'패턴문자열' -> content에서 패턴문자열을 찾아 리스트로 리턴
    # #으로 시작해서 
    # ^ : 뒤에 문자들이 없어야 함 (whitespace, #, 콤마(,), 역슬래쉬(\)) + 글자들
    # \s : whitespace / 
    
    
    # 작성일자 정보
    date = soup.select('time.FH9sR.Nzb55')[0]['datetime'][:10]
    
    # 좋아요
    try:
        like = soup.select('div.Nm9Fw > a > span')[0].text
    except:
        like = 0
    
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    
    data = [content, date, like, place, tags]
    return data


def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

In [16]:
# 300개 크롤링하기

results = [ ]

target = 300      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로)  2초 대기 후, 다음 게시글로 넘어가도록 try, except 구문 활용
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(1)
        move_next(driver)
    

results[0:2]

[['💚인왕산 성공적💚.최근 들어 제일 행복했던 순간.등산 후 마스크 살짝 벗고 숨쉴 때그렇게 상쾌하며 행복할 수가 없더라고요👍찐 행복한 웃음😊.정말 인왕산 찐 추천입니다!특히 오후 등산 추천👍😊👍.같이 가실 분 있나 혹쉬?.#서울등산#서울산#인왕산#인왕산정상#인왕산자락길#인왕산둘레길#인왕산숲길#인왕산등산#서울사진맛집#종로여행#국내여행#여행에미치다',
  '2020-04-26',
  '480',
  '인왕산',
  ['#서울등산',
   '#서울산',
   '#인왕산',
   '#인왕산정상',
   '#인왕산자락길',
   '#인왕산둘레길',
   '#인왕산숲길',
   '#인왕산등산',
   '#서울사진맛집',
   '#종로여행',
   '#국내여행',
   '#여행에미치다']],
 ['.💜 #포토시그니처_익선동한옥거리점 💜포토시그니처와 산리오캐릭터 프레임을 출시했다!!재미있게 사진도 찍고 사랑스러운 산리오캐릭터도 소장할 수 있는 절호의 기회(๑>ᴗ<๑)6월 한달 한정으로 만나는사랑스러운 디자인의 캐릭터들을 지금 만나러 오세요.✔서울특별시 종로구 수표로28길 17-6(익선동)24h 포토시그니처_익선동한옥거리점✔배경색과 산리오 캐릭터 꿀조합도 모았으니 참고!#포토시그니처 #셀프스튜디오 #커플사진 #네컷포즈 #셀프촬영#인생샷 #우정사진 #데이트코스 #무인사진관 #실내데이트 #셀프사진#헬로키티 #사진 #인생사진 #이색데이트 #주말데이트 #키티덕후 #키티스타그램 #셀스타그램 #익선동 #익선동한옥거리 #익선동포토시그니처 #익선동데이트 #익선동골목 #익선동핫플 #종로데이트 #종로익선동 #종로여행 #종로핫플',
  '2021-06-09',
  '390',
  '',
  ['#포토시그니처_익선동한옥거리점',
   '#포토시그니처',
   '#셀프스튜디오',
   '#커플사진',
   '#네컷포즈',
   '#셀프촬영',
   '#인생샷',
   '#우정사진',
   '#데이트코스',
   '#무인사진관',
   '#실내데이트',
   '#셀프사진',
   '#헬로키티',


In [17]:
# 크롤링 결과 저장하기 (폴더 있어야 오류 안남)

import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags'] #컬럼 지정 
results_df.to_excel('./210717_crawling_jr2.xlsx', index = False)